In [1]:
import os
import util
import numpy as np
import pandas as pd
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Input, Conv3D, MaxPool3D, GlobalAveragePooling3D, Flatten, Dense, Dropout, Rescaling, BatchNormalization
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping

# Load Data

In [2]:
base_dir = 'artifacts'
data = os.path.join(base_dir, 'split_3d_data.pkl')
batch_size = 8

In [3]:
X_train, X_val, X_test, y_train, y_val, y_test = util.load_split_3d_data(data)

In [4]:
X_train.shape, X_test.shape, X_val.shape

((280, 16, 224, 224, 3), (60, 16, 224, 224, 3), (60, 16, 224, 224, 3))

In [5]:
num_frames, img_size = X_train.shape[1], X_train.shape[2:4]

In [6]:
y_train.shape, y_test.shape, y_val.shape

((280,), (60,), (60,))

## Factorizing Target

In [7]:
y_val

array(['real', 'real', 'real', 'real', 'real', 'real', 'real', 'real',
       'real', 'real', 'real', 'real', 'real', 'real', 'real', 'real',
       'real', 'real', 'real', 'real', 'real', 'real', 'real', 'real',
       'real', 'real', 'real', 'real', 'real', 'real', 'fake', 'fake',
       'fake', 'fake', 'fake', 'fake', 'fake', 'fake', 'fake', 'fake',
       'fake', 'fake', 'fake', 'fake', 'fake', 'fake', 'fake', 'fake',
       'fake', 'fake', 'fake', 'fake', 'fake', 'fake', 'fake', 'fake',
       'fake', 'fake', 'fake', 'fake'], dtype='<U4')

In [8]:
pd.factorize(y_val)[0]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], dtype=int64)

In [9]:
labels = pd.factorize(y_val)[1]
print(labels)

['real' 'fake']


In [10]:
y_train, y_val, y_test = pd.factorize(y_train)[0], pd.factorize(y_val)[0], pd.factorize(y_test)[0]

In [11]:
y_train.shape, y_test.shape, y_val.shape

((280,), (60,), (60,))

# 3D CNN

In [12]:
(num_frames,)+img_size+(3,)

(16, 224, 224, 3)

In [13]:
model = Sequential(
    [
        Input(shape=(num_frames,)+img_size+(3,)),
        Rescaling(1./255.),
        Conv3D(32, (3,3,3), activation='relu', padding='same'),
        Conv3D(64, (3,3,3), activation='relu', padding='same'),
        MaxPool3D((2,2,2)),
        Conv3D(128, (3,3,3), activation='relu', padding='same'),
        MaxPool3D((2,2,2)),
        Flatten(),
        Dense(128, activation='relu'),
        Dropout(0.5),
        Dense(1, activation='sigmoid')
    ]
)
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ rescaling (Rescaling)                │ (None, 16, 224, 224, 3)     │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv3d (Conv3D)                      │ (None, 16, 224, 224, 32)    │           2,624 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv3d_1 (Conv3D)                    │ (None, 16, 224, 224, 64)    │          55,360 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling3d (MaxPooling3D)         │ (None, 8, 112, 112, 64)     │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv3d_2 (Conv3D)                    │ (None, 8, 112, 112, 128)    │         221,312 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling3d_1 (MaxPooling3D)       │ (None, 4, 56, 56, 128)      │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 1605632)             │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 128)                 │     205,521,024 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 1)                   │             129 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 205,800,449 (785.07 MB)

 Trainable params: 205,800,449 (785.07 MB)

 Non-trainable params: 0 (0.00 B)

In [14]:
model.compile(loss='binary_crossentropy', 
              optimizer=Adam(learning_rate=0.01), 
              metrics=['accuracy'])

In [15]:
estop = EarlyStopping(monitor='val_loss', mode='min',
                      min_delta=1e-5, patience=5,
                      restore_best_weights=True, verbose=1)
model.fit(X_train, y_train, 
          validation_data=(X_val, y_val),
          epochs=500, batch_size=batch_size,
          callbacks=[estop],verbose=1)

Epoch 1/500
35/35 ━━━━━━━━━━━━━━━━━━━━ 270s 7s/step - accuracy: 0.5286 - loss: 774.5608 - val_accuracy: 0.5000 - val_loss: 0.6953
Epoch 2/500
35/35 ━━━━━━━━━━━━━━━━━━━━ 246s 7s/step - accuracy: 0.4464 - loss: 0.6976 - val_accuracy: 0.5000 - val_loss: 0.6932
Epoch 3/500
35/35 ━━━━━━━━━━━━━━━━━━━━ 246s 7s/step - accuracy: 0.5000 - loss: 0.6942 - val_accuracy: 0.5000 - val_loss: 0.6932
Epoch 4/500
35/35 ━━━━━━━━━━━━━━━━━━━━ 245s 7s/step - accuracy: 0.4786 - loss: 0.6940 - val_accuracy: 0.5000 - val_loss: 0.6931
Epoch 5/500
35/35 ━━━━━━━━━━━━━━━━━━━━ 251s 7s/step - accuracy: 0.5000 - loss: 0.6945 - val_accuracy: 0.5000 - val_loss: 0.6934
Epoch 6/500
35/35 ━━━━━━━━━━━━━━━━━━━━ 253s 7s/step - accuracy: 0.5000 - loss: 0.6934 - val_accuracy: 0.5000 - val_loss: 0.6931
Epoch 7/500
35/35 ━━━━━━━━━━━━━━━━━━━━ 243s 7s/step - accuracy: 0.4429 - loss: 0.6938 - val_accuracy: 0.5000 - val_loss: 0.6932
Epoch 8/500
35/35 ━━━━━━━━━━━━━━━━━━━━ 245s 7s/step - accuracy: 0.5000 - loss: 0.6940 - val_accuracy: 

In [16]:
test_loss1, test_accuracy1 = model.evaluate(X_test, y_test)
print("Test accuracy:", test_accuracy1)
print("Test loss:", test_loss1)

2/2 ━━━━━━━━━━━━━━━━━━━━ 34s 11s/step - accuracy: 0.5000 - loss: 0.6931
Test accuracy: 0.5
Test loss: 0.6931473016738892


**A 3D CNN model also didn't perform any better than a random classifier. This is exactly like the baseline 2D CNN model, however, the training time was significantly higher, making this model highly infeasible.**